In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB


import sys,os






In [71]:
import jieba
import numpy as np
import sys,os
tpattern_path = '../../../classifier/models/time_pattern/'
sys.path.append(tpattern_path)
from time_pattern import TimePattern
env_path = '../../../classifier/env/'
sys.path.append(env_path)
from env import ENV
log_path = '../../../classifier/lib/'
sys.path.append(log_path)
from log import Logger





class BaseClassifier:
    def __init__(self, **model):
        """
        suggested parameters:
        svc, logistic, nb, jieba_path,tfidf
        """
        self._load_model(**model)
        self.log = None
        
    def warm_up(self):
        self.other.classify('')
        
    def _load_model(self,**model):
        self.svc = model.get('svc')
        self.logistic = model.get('logistic')
        self.nb = model.get('nb')
        self.tfidf = model.get('tfidf')
        self.other = model.get('other')
        # load jieba
        jieba_path = model.get('jieba_path')
        if jieba_path is not None:
            jieba.load_userdict(jieba_path)
            
    def _ext_time(self,sentence, lower_bounder=36, upper_bounder=24*15):
        """
        time label 0: extract length is 0
        time label 2: extract length is 2
        time label 10: extract length is 1, delta time is within the shortest time
        time label 11: extract length is 1, delta time is within the middle time
        time label 12: extract length is 1, delta time is greater than the longest time
        """
        time_extract = self.re_time.process(sentence)
        time_label = 0
        if len(time_extract) == 0:
            time_label = 0
            self.log.debug('No time was extracted!')
        elif len(time_extract) > 1:
            time_label = 2
            self.log.debug('More than 2 times were extracted!')
        else:
            delta = time_extract[0]['gapH']
            self.log.debug('Just one time was extracted! And the time delta is {} hours'.format(delta))
            if delta < lower_bounder:
                time_label = 10
                self.log.debug('The delta is less than lower bounder {} hours'.format(lower_bounder))
            elif lower_bounder <= delta < upper_bounder:
                time_label = 11
                self.log.debug('The delta is greater than lower bounder {} hours but less than upper bounder {} hours'.format(lower_bounder,upper_bounder))
            else:
                time_label = 12
                self.log.debug('The delta is greater than upper bounder {} hours'.format(upper_bounder))
                
        return {'label':time_label,'time_extract':time_extract}
        
    


class IDClassifier(BaseClassifier):
    
    def __init__(self,**model):
        super().__init__(**model)
        self.label_meaning = 'ifDebtorAnswersing'
        self.label_meaning_map = {0:'y',1:'n'}
       
    def classify(self, sentence,lower_bounder=None,upper_bounder=None,debug=False):
        """
        ML model wrapper. No time regular expression involved!
        input: sentence - type string
        return label
        """
        if self.log is None:
            self.log = Logger(self.__class__.__name__,level=ENV.MODEL_LOG_LEVEL.value).logger
        sentence = jieba.cut(sentence, cut_all = False)
        sentence = ' '.join(sentence)
        matrix = self.tfidf.transform([sentence])
        self.log.debug('In transfered tfidf, the number of words in vocalbulary is: {}'.format(len(matrix.data)))
        result = np.vstack((self.svc.predict_proba(matrix),
                                 self.logistic.predict_proba(matrix),
                                 self.nb.predict_proba(matrix)))
        
        av_pred = np.mean(result, axis = 0)
        max_pred = np.max(av_pred, axis = 0)
        max_arg = np.argmax(av_pred)
        response = None
        label = max_arg
        if label == 2:
            response = self.other.classify(sentence)
            label = response['label']
        if debug:
            dictionary = {'label': label, 'pred_prob': result, 'av_pred': av_pred,'other_response':response}
        else:
            if response is not None:
                response = float(max(response['av_pred']))
            av_pred_value = float(max(av_pred))
            dictionary = {'label': label, 'av_pred': av_pred_value,'other_response':response}
        self.log.debug('Final Pred label is: {}'.format(label))
        dictionary.update({self.label_meaning:self.label_meaning_map.get(label,'null')})
        return dictionary
    
    
    

class IfKnowDebtor(BaseClassifier):
    
    def __init__(self,**model):
        super().__init__(**model)
        self.label_meaning = 'ifKnowDebtor'
        self.label_meaning_map = {0:'y',1:'n'}
        
        
    def classify(self, sentence,lower_bounder=None,upper_bounder=None,debug=False):
        """
        ML model wrapper. No time regular expression involved!
        input: sentence - type string
        return label
        """
        if self.log is None:
            self.log = Logger(self.__class__.__name__,level=ENV.MODEL_LOG_LEVEL.value).logger
        sentence = jieba.cut(sentence, cut_all = False)
        sentence = ' '.join(sentence)
        matrix = self.tfidf.transform([sentence])
        self.log.debug('In transfered tfidf, the number of words in vocalbulary is: {}'.format(len(matrix.data)))
        result = np.vstack((self.svc.predict_proba(matrix),
                                 self.logistic.predict_proba(matrix),
                                 self.nb.predict_proba(matrix)))
        
        av_pred = np.mean(result, axis = 0)
        max_pred = np.max(av_pred, axis = 0)
        max_arg = np.argmax(av_pred)
        response = None
        label = max_arg
        if label == 2:
            response = self.other.classify(sentence)
            label = response['label']
        if debug:
            dictionary = {'label': label, 'pred_prob': result, 'av_pred': av_pred,'other_response':response}
        else:
            if response is not None:
                response = float(max(response['av_pred']))
            av_pred_value = float(max(av_pred))
            dictionary = {'label': label, 'av_pred': av_pred_value,'other_response':response}
        
        self.log.debug('Final Pred label is: {}'.format(label))
        dictionary.update({self.label_meaning:self.label_meaning_map.get(label,'null')})
        return dictionary
    
    
    

    
class ConfirmLoan(BaseClassifier):
    
    def __init__(self,**model):
        super().__init__(**model)
        self.re_time = TimePattern()
        self.label_meaning = 'ifAdmitLoan'
        self.label_meaning_map = {0:'y',1:'n'}
        
    def classify(self, sentence,lower_bounder=36, upper_bounder=72,debug=False):
        """
        if len(time_extract) == 0 --> run through ML
        if len(time_extract) == 1(within short time) --> jump to n103
            other --> jump to n15
        """
        if self.log is None:
            self.log = Logger(self.__class__.__name__,level=ENV.MODEL_LOG_LEVEL.value).logger
        time_result = self._ext_time(sentence,lower_bounder, upper_bounder)
        time_label = time_result['label']
        time_extract = time_result['time_extract']
        # remove time pattern from setence
        sentence = self.re_time.remove_time(sentence)
        sentence = jieba.cut(sentence, cut_all = False)
        sentence = ' '.join(sentence)
        matrix = self.tfidf.transform([sentence])
        self.log.debug('In transfered tfidf, the number of words in vocalbulary is: {}'.format(len(matrix.data)))
        result = np.vstack((self.svc.predict_proba(matrix),
                                 self.logistic.predict_proba(matrix),
                                 self.nb.predict_proba(matrix)))
        
        av_pred = np.mean(result, axis = 0)
        max_pred = np.max(av_pred, axis = 0)
        max_arg = np.argmax(av_pred)
        response = None
        label = max_arg
        if label == 2:
            response = self.other.classify(sentence)
            label = response['label']
        
        # interact with regular expression
        if (time_label == 10) and (label != 1):
            label = 10
    
        if debug:
            dictionary = {'label': label, 'pred_prob': result, 'av_pred': av_pred,'other_response':response}
        else:
            if response is not None:
                response = float(max(response['av_pred']))
            av_pred_value = float(max(av_pred))
            dictionary = {'label': label, 'av_pred': av_pred_value,'other_response':response}
        dictionary.update({'timeExtract':time_extract})
        self.log.debug('Final Pred label is: {}'.format(label))
        dictionary.update({self.label_meaning:self.label_meaning_map.get(label,'null')})
        return dictionary
    
    
    

    
class WillingToPay(BaseClassifier):
    def __init__(self,**model):
        super().__init__(**model)
        self.re_time = TimePattern()
        self.label_meaning = 'ifWillingToPay'
        self.label_meaning_map = {0:'y',1:'n'}
    
        
        
    def classify(self, sentence, lower_bounder=36, upper_bounder=72,debug=False):
        """
        0 - high willing to pay (ML + Reg, between short and long)
        1 - not willing to pay (ML + Reg, too long)
        2 - hope to cut
        3 - other
        Re:
        if time len(extract) >=2, and the min time is within the tolerance --> connect to self and confirm which day to pay,
                                    output label is 10
        if time len(extract) ==1, and the min time is within the tolerance --> run through ML
                                    and the min time is within the middle time --> not run ML, connect to self, output label 1,
                                    and the min time is longer than the longest time --> no ML, connect to self,output1 sentiment +1
        """
        if self.log is None:
            self.log = Logger(self.__class__.__name__,level=ENV.MODEL_LOG_LEVEL.value).logger
        dictionary = {}
        # Regular expression
        time_result = self._ext_time(sentence,lower_bounder, upper_bounder)
        time_label = time_result['label']
        time_extract = time_result['time_extract']
        response = None
        if time_label == 2:
            min_time = time_extract[0]['gapH']
            for each in time_extract[1:]:
                _time = each['gapH']
                if _time < min_time:
                    min_time = _time
            if min_time <= lower_bounder:
                self.log.debug('There are more than 1 time extracted. And the min {} hours is shorter than lower bounder! The output label is set to 10!'.format(min_time))
                return {'label':10 , 'pred_prob': 1.0, 'av_pred': 1.0, 'time_extract':time_extract}
        else:           
            # ML model process
            # remove time pattern from setence
            sentence = self.re_time.remove_time(sentence)
            sentence = jieba.cut(sentence, cut_all = False)
            sentence = ' '.join(sentence)
            matrix = self.tfidf.transform([sentence])
            self.log.debug('In transfered tfidf, the number of words in vocalbulary is: {}'.format(len(matrix.data)))
            result = np.vstack((self.svc.predict_proba(matrix),
                                     self.logistic.predict_proba(matrix),
                                     self.nb.predict_proba(matrix)))

            av_pred = np.mean(result, axis = 0)
            max_pred = np.max(av_pred, axis = 0)
            max_arg = np.argmax(av_pred)
            label = max_arg
            

            if label == 3:
                response = self.other.classify(sentence)
                label = response['label']
            # interact with regular expression
            if (time_label == 2) and (label != 1):
                label = 10
            
            ####### interact with Regular expression
            if time_label == 11:
                label = 1
            elif time_label == 12:
                label = 1
                dictionary.update({'add_sentiment':1})
            if debug:
                dictionary = {'label': label, 'pred_prob': result, 'av_pred': av_pred,'other_response':response}
            else:
                if response is not None:
                    response = float(max(response['av_pred']))
                av_pred_value = float(max(av_pred))
                dictionary = {'label': label, 'av_pred': av_pred_value,'other_response':response}
            dictionary.update({'timeExtract':time_extract})
            self.log.debug('Final Pred label is: {}'.format(label))
            dictionary.update({self.label_meaning:self.label_meaning_map.get(label,'null')})
            return dictionary
    
    
    

class CutDebt(BaseClassifier):
    def __init__(self,**model):
        super().__init__(**model)
        self.re_time = TimePattern()
        self.label_meaning = 'ifAcceptCutDebt'
        self.label_meaning_map = {0:'y',1:'n'}
        
    def classify(self, sentence,lower_bounder=36, upper_bounder=72,debug=False):
        """
        Re:
        if time len(extract) >=2, and the min time is within the tolerance --> connect to self and confirm which day to pay,
                                    output label is 10
        if time len(extract) ==1, and the min time is within the tolerance --> run through ML
                                    and the min time is within the middle time --> not run ML, connect to self, output label 1,
                                    and the min time is longer than the longest time --> no ML, connect to self,output1 sentiment +1
        """
        if self.log is None:
            self.log = Logger(self.__class__.__name__,level=ENV.MODEL_LOG_LEVEL.value).logger
        dictionary = {}
        # Regular expression
        time_result = self._ext_time(sentence,lower_bounder, upper_bounder)
        time_label = time_result['label']
        time_extract = time_result['time_extract'] 
        response = None
        if time_label == 2:
            min_time = time_extract[0]['gapH']
            for each in time_extract[1:]:
                _time = each['gapH']
                if _time < min_time:
                    min_time = _time
            if min_time <= lower_bounder:
                self.log.debug('There are more than 1 time extracted. And the min {} hours is shorter than lower bounder! The output label is set to 10!'.format(min_time))
                return {'label':10 , 'pred_prob': 1.0, 'av_pred': 1.0, 'time_extract':time_extract}
        else:
            # remove time pattern from setence
            sentence = self.re_time.remove_time(sentence)
            sentence = jieba.cut(sentence, cut_all = False)
            sentence = ' '.join(sentence)
            matrix = self.tfidf.transform([sentence])
            self.log.debug('In transfered tfidf, the number of words in vocalbulary is: {}'.format(len(matrix.data)))
            result = np.vstack((self.svc.predict_proba(matrix),
                                     self.logistic.predict_proba(matrix),
                                     self.nb.predict_proba(matrix)))

            av_pred = np.mean(result, axis = 0)
            max_pred = np.max(av_pred, axis = 0)
            max_arg = np.argmax(av_pred)
            label = max_arg
            if label == 2:
                response = self.other.classify(sentence)
                label = response['label']

            dictionary = {'label': label, 'pred_prob': result, 'av_pred': av_pred}
            # interact with regular expression
            if (time_label == 2) and (label != 1):
                label = 10
            
            ####### interact with Regular expression
            if time_label == 11:
                label = 1
            elif time_label == 12:
                label = 1
                dictionary.update({'add_sentiment':1})
            if debug:
                dictionary = {'label': label, 'pred_prob': result, 'av_pred': av_pred,'other_response':response}
            else:
                if response is not None:
                    response = float(max(response['av_pred']))
                av_pred_value = float(max(av_pred))
                dictionary = {'label': label, 'av_pred': av_pred_value,'other_response':response}
            dictionary.update({'timeExtract':time_extract})
            self.log.debug('Final Pred label is: {}'.format(label))
            dictionary.update({self.label_meaning:self.label_meaning_map.get(label,'null')})
            return dictionary
    
    
    
class Installment(BaseClassifier):
    def __init__(self,**model):
        super().__init__(**model)
        self.re_time = TimePattern()
        self.label_meaning = 'ifAcceptInstallment'
        self.label_meaning_map = {0:'y',1:'n'}
        
        
    def classify(self, sentence,lower_bounder=36, upper_bounder=72,debug=False):
        """
        Re:
        if time len(extract) >=2, and the min time is within the tolerance --> connect to self and confirm which day to pay,
                                    output label is 10
        if time len(extract) ==1, and the min time is within the tolerance --> run through ML
                                    and the min time is within the middle time --> not run ML, connect to self, output label 1,
                                    and the min time is longer than the longest time --> no ML, connect to self,output1 sentiment +1
        """
        if self.log is None:
            self.log = Logger(self.__class__.__name__,level=ENV.MODEL_LOG_LEVEL.value).logger
        dictionary= {}
        # Regular expression
        time_result = self._ext_time(sentence,lower_bounder, upper_bounder)
        time_label = time_result['label']
        time_extract = time_result['time_extract']    
        response = None
        if time_label == 2:
            min_time = time_extract[0]['gapH']
            for each in time_extract[1:]:
                _time = each['gapH']
                if _time < min_time:
                    min_time = _time
            if min_time <= lower_bounder:
                self.log.debug('There are more than 1 time extracted. And the min {} hours is shorter than lower bounder! The output label is set to 10!'.format(min_time))
                return {'label':10 , 'pred_prob': 1.0, 'av_pred': 1.0, 'time_extract':time_extract}
        else:
            # remove time pattern from setence
            sentence = self.re_time.remove_time(sentence)
            sentence = jieba.cut(sentence, cut_all = False)
            sentence = ' '.join(sentence)
            matrix = self.tfidf.transform([sentence])
            self.log.debug('In transfered tfidf, the number of words in vocalbulary is: {}'.format(len(matrix.data)))
            result = np.vstack((self.svc.predict_proba(matrix),
                                     self.logistic.predict_proba(matrix),
                                     self.nb.predict_proba(matrix)))

            av_pred = np.mean(result, axis = 0)
            max_pred = np.max(av_pred, axis = 0)
            max_arg = np.argmax(av_pred)
            label = max_arg

            if label == 2:
                response = self.other.classify(sentence)
                label = response['label']

            dictionary = {'label': label, 'pred_prob': result, 'av_pred': av_pred}
            ####### interact with Regular expression
            if time_label == 11:
                label = 1
            elif time_label == 12:
                label = 1
                dictionary.update({'add_sentiment':1})
            if debug:
                dictionary = {'label': label, 'pred_prob': result, 'av_pred': av_pred,'other_response':response}
            else:
                if response is not None:
                    response = float(max(response['av_pred']))
                av_pred_value = float(max(av_pred))
                dictionary = {'label': label, 'av_pred': av_pred_value,'other_response':response}
            dictionary.update({'timeExtract':time_extract})
            self.log.debug('Final Pred label is: {}'.format(label))
            dictionary.update({self.label_meaning:self.label_meaning_map.get(label,'null')})
            return dictionary



class ClassifierOther:
    def __init__(self, **model):
        """
        suggested parameters:
        svc, logistic, nb, jieba_path, tfidf
        """
        self.log = None
        self._load_model(**model)
        self._load_attributes(**model)
        
        
    def _load_model(self,**model):
        self.svc = model.get('svc')
        self.logistic = model.get('logistic')
        self.nb = model.get('nb')
        self.tfidf = model.get('tfidf')
        # load jieba
        jieba_path = model.get('jieba_path')
        if jieba_path is not None:
            jieba.load_userdict(jieba_path)
        
            
    def _load_attributes(self, **model):
        self.label_mapping = model.get('possible_label')
        self.label_mapping = sorted(list(set(self.label_mapping)))
        
    
    def classify(self, sentence):
        """
        input: sentence
        output: result(dictionary)
        """
        if self.log is None:
            self.log = Logger(self.__class__.__name__,level=ENV.MODEL_LOG_LEVEL.value).logger
        sentence = jieba.cut(sentence, cut_all = False)
        sentence = ' '.join(sentence)
        matrix = self.tfidf.transform([sentence])
        self.log.debug('In transfered tfidf, the number of words in vocalbulary is: {}'.format(len(matrix.data)))
        
        result = np.vstack((self.svc.predict_proba(matrix),
                                 self.logistic.predict_proba(matrix),
                                 self.nb.predict_proba(matrix)))
        
        av_pred = np.mean(result, axis = 0)
        max_pred = np.max(av_pred, axis = 0)
        max_arg = np.argmax(av_pred)
        
        label = max_arg
        label = self.label_mapping[label]
            
        dictionary = {'label': label, 'pred_prob': result, 'av_pred': av_pred}
        self.log.debug('Possible labels are: {}'.format(self.label_mapping))
        self.log.debug('Other- Final Pred label is: {}'.format(dictionary['label']))
        self.log.debug('Other- svc,logistic,nb result:\n {}'.format(dictionary['pred_prob']))
        self.log.debug('Other- ave result:\n {}'.format(dictionary['av_pred']))
        return dictionary

In [72]:
model_list = {
                'IDClassifier':IDClassifier, 
                  'CutDebt':CutDebt, 
                  'WillingToPay':WillingToPay,
                  'IfKnowDebtor':IfKnowDebtor,
                  'Installment':Installment,
                  'ConfirmLoan':ConfirmLoan}

In [73]:
#encoding=utf-8
import pandas as pd
import numpy as np
import re
import string

import jieba
jieba.load_userdict("../../code/WordCut/userdict.txt")

import gc
import pickle
path = '../../data/others/'

# load cleaned other data
other_non109 = pd.read_csv('../../data/others/cleaned_mock_up_data_non109.csv')
other_109 = pd.read_csv('../../data/others/cleaned_mock_up_data_109.csv')
strategy_mat = pd.read_csv(path + 'strategy_mat.csv', encoding='utf8')

print(other_non109.shape)
print(other_109.shape)
print(strategy_mat.shape)

(3950, 2)
(1408, 8)
(16, 9)


In [74]:
def get_other_data(df_non109,df_109,strategy_mat,classifier):
    possible_label = sorted(list(set(strategy_mat[strategy_mat[classifier]==0]['label'].values)))
    train_data_non109 = df_non109[df_non109['label'].apply(lambda x: x in possible_label)]
    train_data_109 = df_109[df_109[classifier]==0]
    data = pd.concat([train_data_non109,train_data_109],ignore_index=True,sort=True)
    return data

def train_other_model(other_data,save_path,model):
    phrase_vectorizer_other = TfidfVectorizer(ngram_range=(1,3),
                                strip_accents='unicode', 
                                max_features=100000, 
                                analyzer='word',
                                sublinear_tf=True,
                                token_pattern=r'\w{1,}')

    print('fitting phrase')
    phrase_vectorizer_other.fit(other_data.text)

    print('transform phrase')
    phrase = phrase_vectorizer_other.transform(other_data.text)


    # linear svc
    l_svc = LinearSVC()
    lsvc = CalibratedClassifierCV(l_svc) 
    lsvc.fit(phrase, other_data.label)


    # logistic
    log_r = LogisticRegression()
    log_r.fit(phrase, other_data.label)


    # Naive Bayes
    naive_b = MultinomialNB()
    naive_b.fit(phrase, other_data.label)
    
    print('finish training others')
    
    
    # other wrapper 
    other_model = ClassifierOther(svc=lsvc, logistic=log_r, nb=naive_b, tfidf=phrase_vectorizer_other, jieba_path='../WordCut/userdict.txt',possible_label=lsvc.classes_)
    
    # Saving:
    evl_path = save_path.format(model,model)
    print('saving to path: {}'.format(evl_path))
#     pickle.dump(other_model, open(evl_path, "wb"))
    return other_model
    
    
def train_main_model(df,save_path,model,other_model):
    # get tfidf
    
    phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

    print('fitting phrase')
    phrase_vectorizer.fit(df.split_text)

    print('transform phrase')
    phrase = phrase_vectorizer.transform(df.split_text)
    
    # linear svc
    l_svc = LinearSVC()
    lsvc = CalibratedClassifierCV(l_svc) 
    lsvc.fit(phrase, df.label)
    
    
    # logistic
    log_r = LogisticRegression()
    log_r.fit(phrase, df.label)
    
    
    # Naive Bayes
    naive_b = MultinomialNB()
    naive_b.fit(phrase, df.label)
    print('finish training')
    
    main_model = model_list[model](svc=lsvc, logistic=log_r, nb=naive_b, tfidf=phrase_vectorizer, other=other_model,  jieba_path='../WordCut/userdict.txt')
    evl_path = save_path.format(model,model)
#     pickle.dump(main_model, open(evl_path, "wb"))
    print('saving to path: {}'.format(evl_path))
    return main_model
    
    

# Train Other Model + Main Model

# Train IDClassifier

In [83]:
# each_model = 'IDClassifier' 
save_path_other = '../../../classifier/saved_model/{}/other_flow/{}.pkl'
save_path_main = '../../../classifier/saved_model/{}/main_flow/{}.pkl'

each_model = 'Installment'
other_data = get_other_data(other_non109,other_109,strategy_mat,each_model)
other_data = other_data.sample(frac=1,random_state=19)

other_model = train_other_model(other_data,save_path_other,each_model)

# train main
df_main = pd.read_csv('../../data/{}/cleaned_mock_up_data.csv'.format(each_model))
other_label = int(max(set(df_main.label)) + 1)
ava_others = other_data.rename({'text':'split_text'},axis=1)
ava_others['label'] = other_label
df_main = pd.concat([df_main,ava_others],sort=True)
df_main = df_main.sample(frac=1,random_state=6).reset_index(drop=True)
print('=====  {} ======='.format(each_model))
print(df_main.label.value_counts())
clf = train_main_model(df_main,save_path_main,each_model,other_model)
print('\n')

fitting phrase
transform phrase
finish training others
saving to path: ../../../classifier/saved_model/Installment/other_flow/Installment.pkl
=====  Installment =======
2    4703
0    3726
1    2375
Name: label, dtype: int64
fitting phrase
transform phrase
finish training
Time Zone is set from ENV: Asia/Shanghai
saving to path: ../../../classifier/saved_model/Installment/main_flow/Installment.pkl




In [86]:
clf.classify('后天下午2点就还')

{'label': 1,
 'av_pred': 0.9679038176105955,
 'other_response': None,
 'timeExtract': [{'pattern': '后天下午2点',
   'time': datetime.datetime(2018, 8, 22, 14, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
   'gapS': 182356.032478,
   'gapH': 50.654453466111114}],
 'ifAcceptInstallment': 'n'}